In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import shutil
import csv
import os

In [18]:
# 輸出 1.tmp + hr 的圖表 2.medians 的圖表
# 主要函式
# path為要讀取的資料夾 此資料夾包含所有病患的資料夾
# time_step為時間間隔 單位為分鐘 請輸入20 or 30 ...
def box_plot(path, time_step, start = 0):
    
#     plot, medians, mat檔要存的地方 個別建立一個新的資料夾
    plot_path = path + '_plot' + '(' + str(time_step) + 'mins)'
    med_path = path + '_medians' + '(' + str(time_step) + 'mins)'
#     mat_path = path + '_mat' + '(' + str(time_step) + 'mins)'
    check_dir(plot_path)
    check_dir(med_path)
#     check_dir(mat_path)

    pid_list = os.listdir(path) 
    for pid in pid_list: # 進入每一個資料夾
        if 'check' in pid: continue
        path2 = path + '/' + pid
        
        check_csv_num(path2)        
        for file in os.listdir(path2):
            if 'check' in file: continue
            if 'xlsx' in file: continue
            os.mkdir( plot_path + '/' + pid) # 一個人的plot圖分別存在一個新的資料夾
            os.mkdir( med_path + '/' + pid) # 一個人的medians圖分別存在一個新的資料夾
#             os.mkdir( mat_path + '/' + pid) # 一個人的mat圖分別存在一個新的資料夾

#             將medians, q1, q3分別記錄下來 並存成csv
            tmp_q1 = []
            tmp_med = []
            tmp_q3 = []
            hr_q1 = []
            hr_med = []
            hr_q3 = []
            num = []
            date_num = []
            
#             dataframe的標題分別對應到的值
#             0  1    2    3  4    5 
#             ID date time hp temp move

            df = pd.read_csv( path + '/' + pid + '/' + file, header = None, skiprows=1)
            df[1] = [datetime.datetime.strptime(i, '%Y-%m-%d') for i in df[1].values]
            df[2] = [datetime.datetime.strptime(i, '%H:%M:%S').time() for i in df[2].values]
            date_list = list(dict.fromkeys(df[1].values))
    #             dataframe的標題分別對應到的值
    #             0  1    2    3  4    5 
    #             ID date time hr temp move

            for d_idx in range(len(date_list)):
                date = df[1].values
                today_date = date_list[d_idx]
                d_filter = (date>=date_list[d_idx])&(date<(date_list[d_idx]+np.timedelta64(1,'D')))
                time_zone = 'From '+np.datetime_as_string(date_list[d_idx], unit = 'D')+' to '+np.datetime_as_string(date_list[d_idx]+np.timedelta64(1,'D'), unit = 'D')

                TMP = (df[4].values)[d_filter] # each temp 
                HP = (df[3].values)[d_filter] # each heart rate
                time = (df[2].values)[d_filter] # each hour

                TMP = [i/100 for i in TMP]
                TMP = np.array(TMP) # 將溫度換算成小數點

                start_datetime = datetime.datetime.combine(pd.to_datetime(today_date), datetime.time(start,0))
                end_datetime = datetime.datetime.combine(pd.to_datetime(today_date), time[-1])
                datetime_list = [(start_datetime+i*datetime.timedelta(minutes = time_step)) for i in range(int(np.floor((end_datetime-start_datetime)/datetime.timedelta(minutes = time_step))))]
    #             datetime_list 為x軸所標記的 當天的所有時間點 00:00, 00:10, 00:20...

                each_tmp = []
                each_hr = []

    #             將各時段分別的TMP和HR紀錄進去each_tmp,each_hr
                for h_idx in range(len(datetime_list)):
                    h_filter = (time>=datetime_list[h_idx].time())&(time<(datetime_list[h_idx]+datetime.timedelta(minutes = time_step)).time())
                    hp = HP[(h_filter)&(HP>=50)&(HP<200)]
                    tmp = TMP[(h_filter)&(TMP>25)&(TMP<45)]
                    each_tmp.append(tmp)
                    each_hr.append(hp)

                if time_step == 10 :
                    time_tag = [datetime.datetime.strftime(datetime_list[i], "%H:%M") if i% 3 == 0 else '' for i in range(len(datetime_list))]
                else :
                    time_tag = [datetime.datetime.strftime(datetime_list[i], "%H:%M") for i in range(len(datetime_list))]

    #             開始畫圖
                plt.figure(figsize = (20,10))
                plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.3)

    #             temp pic
                plt.subplot(211)
                tmp_boxplot = plt.boxplot(each_tmp, flierprops = dict(markerfacecolor = 'b', marker = '.'))
                baseline = plt.plot(np.linspace(1, len(datetime_list), len(datetime_list)), [35 for i in range(len(datetime_list))], 'g:')
                plt.legend(baseline, ['baseline'], loc='upper right')

                plt.xticks(np.linspace(1, len(datetime_list), len(datetime_list)), time_tag, rotation='vertical')
                plt.ylim((29,41))
                plt.xlabel('Hours')
                plt.ylabel('Temperature')
                plt.title(time_zone)

    #             hp pic
                plt.subplot(212)
                hr_boxplot = plt.boxplot(each_hr, flierprops = dict(markerfacecolor = 'b', marker = '.'))
                plt.plot(np.linspace(1, len(datetime_list), len(datetime_list)), [100 for i in range(len(datetime_list))], 'g:')
                plt.legend(baseline, ['baseline'], loc='upper right')
                plt.xticks(np.linspace(1, len(datetime_list), len(datetime_list)), time_tag, rotation='vertical')
                plt.ylim((49,151))
                plt.xlabel('Hours')
                plt.ylabel('Heartbeat') 

                plt.savefig( plot_path + '/' +  pid + '/' +  file[:4] +'_tmp + hr box plot '+time_zone+'.png') # 存檔名稱
                plt.clf()

                for q_idx in range(len(tmp_boxplot['medians'])):

                    tmp_q1.append(round(tmp_boxplot['whiskers'][2*q_idx].get_ydata()[0], 2))
                    tmp_med.append(round(tmp_boxplot['medians'][q_idx].get_ydata()[0], 2))
                    tmp_q3.append(round(tmp_boxplot['whiskers'][2*q_idx+1].get_ydata()[0], 2))

                    hr_q1.append(round(hr_boxplot['whiskers'][2*q_idx].get_ydata()[0], 2))
                    hr_med.append(round(hr_boxplot['medians'][q_idx].get_ydata()[0], 2))
                    hr_q3.append(round(hr_boxplot['whiskers'][2*q_idx+1].get_ydata()[0], 2))

                    num.append(q_idx+1)
                    date_num.append(str(pd.to_datetime(today_date).date()))

            med_dic = {'Date': np.array(date_num), 'Num': np.array(num),
                        'Tmp_q1': np.array(tmp_q1), 'Tmp_med': np.array(tmp_med), 'Tmp_q3': np.array(tmp_q3),
                        'HR_q1': np.array(hr_q1), 'HR_med': np.array(hr_med), 'HR_q3': np.array(hr_q3), }
    #         存成csv檔
            pd.DataFrame(med_dic).to_csv( med_path + '/' + pid + '/' + file[:4] + '_medians.csv', index=False)
    #         存成mat檔
    #         sio.savemat(mat_path + '/' + pid[:4] + '_medians.mat', file_dic) 

# 重建資料夾的函式
def check_dir(path):
    if os.path.isdir( path) :
        shutil.rmtree( path , ignore_errors=True)
    os.mkdir( path ) 
    
# 有幾個csv檔 如有兩個以上 則將其合併成一個輸出
def check_csv_num(path):
    csv_num = 0 # 有幾個csv檔
    for i in os.listdir(path):
        if 'csv' in i: csv_num+=1
    if csv_num>1:# 如果有兩個以上的csv 則將所有的csv彙整成一個csv
        gather_csv(path)

In [19]:
import warnings
warnings.filterwarnings('ignore')

box_plot('ICU20200211', 10)

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

...

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

In [5]:
# 當遇到一個檔案裡面有兩個以上的csv時 就呼叫它
# 這裡的path需要打上病人的資料夾名稱
# ex: path = 'ICU20200211/132'
#     gather_csv(path)

def gather_csv(path):
    pid_list = os.listdir(path)
    pt_info = []
    watch_id = ''
    
    pid_list = [i for i in pid_list if ('check' or 'xlsx') not in i]
    # remove useless file
    
    for pid in pid_list:
        watch_id = pid[:4]
        with open( path + '/' + pid , 'r') as f:
            for line in f.readlines():
                line = line.strip('\n')
                line = line.split(',')
                pt_info.append(line)
        os.remove(path+'/'+pid)
        
    df = pd.DataFrame(pt_info)
    df_list = list(df.drop_duplicates(1)[1]) #用來找日期 等等才能命名
    date_list = []
    
    for i in range(len(df_list)):
        date_list.append(dt.date.today() - pd.to_datetime(df_list[i], format='%Y-%m-%d').date())
    first = df_list[date_list.index(max(date_list))]
    end = df_list[date_list.index(min(date_list))]
    
    with open( path + '/' + watch_id + '_' + first + ',' + end + '.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(pt_info)